# Pulling Venue Data from FourSquare API

__Skip to the Good Stuff__

[Importing Libraries](#Importing-Libraries)

[Connecting to FourSquare](#Connecting-to-FourSquare)

[Testing the Code](#Testing-the-Code)

[Making the DataFrame](#Making-the-DataFrame)

[Dropping the Nulls](#Dropping-the-Nulls)

[Sorting by Venue and Distance](#Sorting-by-Venue-and-Distance)

[Obtaining Neighborhood for Each Arcade](#Obtaining-Neighborhood-for-Each-Arcade)

[Pulling the Ratings](#Pulling-the-Ratings)

### Importing Libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

! pip install folium==0.5.0
import folium # plotting library

import json

pd.set_option('display.max_rows', None) #To view the full dataframe

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [2]:
ny_df = pd.read_csv('newyork_geodata.csv')
print(len(ny_df))
ny_df.head(10)

306


,Borough,Borough_Lat,Borough_Lng,Neighborhood,Hood_Lat,Hood_Lng
0,Bronx,40.846651,-73.878594,Wakefield,40.894705,-73.847201
1,Bronx,40.846651,-73.878594,Co-op City,40.874294,-73.829939
2,Bronx,40.846651,-73.878594,Eastchester,40.887556,-73.827806
3,Bronx,40.846651,-73.878594,Fieldston,40.895437,-73.905643
4,Bronx,40.846651,-73.878594,Riverdale,40.890834,-73.912585
5,Bronx,40.846651,-73.878594,Kingsbridge,40.881687,-73.902818
6,Manhattan,40.789624,-73.959894,Marble Hill,40.876551,-73.910660
7,Bronx,40.846651,-73.878594,Woodlawn,40.898273,-73.867315
8,Bronx,40.846651,-73.878594,Norwood,40.877224,-73.879391
9,Bronx,40.846651,-73.878594,Williamsbridge,40.881039,-73.857446


### Connecting to FourSquare

In [3]:
CLIENT_ID = '4TJ1SLTDO1KADL553GXCDXOJXZONZYTBIRQMVBQ32JYAU3EA' # your Foursquare ID
CLIENT_SECRET = 'TC1N45PW5LSGVLIZ02DDPA2JOMWB2I3BA33OPX55VW2LBKBF' # your Foursquare Secret
ACCESS_TOKEN = 'NS3WWURVAWWAGWSTUXQDQCS0150IPNQIYNHVE5JM1OSX2CAV' # your FourSquare Access Token

In [4]:
VERSION = '20180604'
LIMIT = 30

### Testing the Code

In [5]:
#assign variables for the search
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


In [9]:
for index, hood in enumerate(ny_df['Hood_Lat']):
    if index == 6: #preselected based on venues in initial search
        latitude= ny_df['Hood_Lat'][index]
        longitude= ny_df['Hood_Lng'][index]
print(latitude, longitude)

40.87655077879964 -73.91065965862981


In [10]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

In [11]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '61435b2c92343a338641ff0b'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': []}}

In [10]:
ident=results['response']['venues'][0]
type(ident)
for dictionary in results['response']['venues']:
    print(dictionary['id'])
    print(dictionary['name'])
    print(dictionary['location']['address'])
    print(dictionary['location']['lat'])
    print(dictionary['location']['lng'])
    print(dictionary['location']['distance'])
    for cat in dictionary['categories']:
        print(cat['id'], cat['name'])

4e752f8e52b1c8e51958e177
Italian Bistro


KeyError: 'address'

### Making the DataFrame

In [12]:
#assign variables for the search
search_query = 'Arcade'
radius = 2000
print(search_query + ' .... OK!')

Arcade .... OK!


In [14]:
ny_venues = pd.DataFrame(columns = ['Hood','Venue_ID','Venue_Name','Venue_Distance','Venue_Address','Venue_Latitude','Venue_Longitude','Category_ID','Category_Name'])
ny_venues.head()

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name


In [15]:
for index, hood in enumerate(ny_df['Hood_Lat']):
    try:
        hood_name = ny_df['Neighborhood'][index]
        latitude = ny_df['Hood_Lat'][index]
        longitude = ny_df['Hood_Lng'][index]
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
        results = requests.get(url).json()
    except:
        print('No results', index)
    try:
        res = results['response']['venues']
    except:
        print('No venues', index)
        continue
    for dictionary in results['response']['venues']:
        try:
            ident= dictionary['id']
        except:
            ident= 'No ID'
        try:
            name = dictionary['name']
        except:
            name = 'No Name'
        try:
            ven_add = dictionary['location']['address']
        except:
            ven_add = 'No Address'
        try:
            ven_lat = dictionary['location']['lat']
        except:
            ven_lat = 'No Latitude'
        try:
            ven_lng = dictionary['location']['lng']
        except:
            ven_lng = 'No Longitude'
        try:
            ven_dist = dictionary['location']['distance']
        except:
            ven_dist = 'No Distance'
        try:
            for cat in dictionary['categories']:
                try:
                    cat_id= cat['id']
                except:
                    cat_id= 'No ID'
                try:
                    cat_name = cat['name']
                except:
                    cat_name = 'No Name'
        except:
                continue
        ny_venues = ny_venues.append({'Venue_Distance':ven_dist,'Hood':hood_name, 'Venue_ID':ident, 'Venue_Name':name,'Venue_Address':ven_add,'Venue_Latitude':ven_lat,'Venue_Longitude':ven_lng,'Category_ID':cat_id,'Category_Name':cat_name}, ignore_index=True)

In [16]:
ny_venues = ny_venues.set_index('Hood')
print(len(ny_venues))
ny_venues.head()

497


,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
Hood,,,,,,,,
University Heights,5ac83d3b0f013c3cc9135100,Arcade Pizza,1612,1611 Saint Nicholas Ave,40.855221,-73.929559,4bf58dd8d48988d1ca941735,Pizza Place
University Heights,4d70445dcbc58cfad8686f72,Washington Heights Arcade and Bingo,2041,1422 Saint Nicholas Ave,40.849365,-73.933151,4bf58dd8d48988d1f1931735,General Entertainment
University Heights,4eee04029911d70cb86f1d4c,Arca De Salvacion,425,1984 Jerome Ave,40.852234,-73.908379,4bf58dd8d48988d132941735,Church
Morris Heights,5ac83d3b0f013c3cc9135100,Arcade Pizza,1165,1611 Saint Nicholas Ave,40.855221,-73.929559,4bf58dd8d48988d1ca941735,Pizza Place
Morris Heights,4d70445dcbc58cfad8686f72,Washington Heights Arcade and Bingo,1146,1422 Saint Nicholas Ave,40.849365,-73.933151,4bf58dd8d48988d1f1931735,General Entertainment


In [17]:
print(len(ny_venues))
ny_venues.to_csv('{}_venue_data.csv'.format(search_query))

497


### Dropping the Nulls

In [19]:
ny_venues = pd.read_csv('{}_venue_data.csv'.format(search_query))
print(search_query)
print(len(ny_venues))
ny_venues.head()

Arcade
497


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,University Heights,5ac83d3b0f013c3cc9135100,Arcade Pizza,1612,1611 Saint Nicholas Ave,40.855221,-73.929559,4bf58dd8d48988d1ca941735,Pizza Place
1,University Heights,4d70445dcbc58cfad8686f72,Washington Heights Arcade and Bingo,2041,1422 Saint Nicholas Ave,40.849365,-73.933151,4bf58dd8d48988d1f1931735,General Entertainment
2,University Heights,4eee04029911d70cb86f1d4c,Arca De Salvacion,425,1984 Jerome Ave,40.852234,-73.908379,4bf58dd8d48988d132941735,Church
3,Morris Heights,5ac83d3b0f013c3cc9135100,Arcade Pizza,1165,1611 Saint Nicholas Ave,40.855221,-73.929559,4bf58dd8d48988d1ca941735,Pizza Place
4,Morris Heights,4d70445dcbc58cfad8686f72,Washington Heights Arcade and Bingo,1146,1422 Saint Nicholas Ave,40.849365,-73.933151,4bf58dd8d48988d1f1931735,General Entertainment


In [20]:
ny_venues = ny_venues[ny_venues != 'Null']
ny_venues = ny_venues[ny_venues != '[]']
ny_venues = ny_venues.dropna()
print(len(ny_venues))
ny_venues = ny_venues.reset_index()
ny_venues = ny_venues.drop(['index'], axis = 1)
ny_venues.head()

497


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,University Heights,5ac83d3b0f013c3cc9135100,Arcade Pizza,1612,1611 Saint Nicholas Ave,40.855221,-73.929559,4bf58dd8d48988d1ca941735,Pizza Place
1,University Heights,4d70445dcbc58cfad8686f72,Washington Heights Arcade and Bingo,2041,1422 Saint Nicholas Ave,40.849365,-73.933151,4bf58dd8d48988d1f1931735,General Entertainment
2,University Heights,4eee04029911d70cb86f1d4c,Arca De Salvacion,425,1984 Jerome Ave,40.852234,-73.908379,4bf58dd8d48988d132941735,Church
3,Morris Heights,5ac83d3b0f013c3cc9135100,Arcade Pizza,1165,1611 Saint Nicholas Ave,40.855221,-73.929559,4bf58dd8d48988d1ca941735,Pizza Place
4,Morris Heights,4d70445dcbc58cfad8686f72,Washington Heights Arcade and Bingo,1146,1422 Saint Nicholas Ave,40.849365,-73.933151,4bf58dd8d48988d1f1931735,General Entertainment


### Sorting by Venue and Distance

In [25]:
ny_sorted = ny_venues.sort_values(['Venue_ID','Venue_Distance'], ascending = True)
ny_sorted.head(30)

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
144,Clinton,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,340,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley
494,Hudson Yards,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,605,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley
167,Midtown,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,992,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley
353,Midtown South,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,1030,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley
188,Murray Hill,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,1581,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley
205,Chelsea,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,1696,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley
125,Lincoln Square,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,1929,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley
444,Flatiron,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,1957,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley
418,Tudor City,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,2160,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley
397,Turtle Bay,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,2205,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley


In [26]:
ny_sorted['Category_Name'].unique()

array(['Bowling Alley', 'Arcade', 'Clothing Store', 'Building', 'Plaza',
       'Road', 'Gaming Cafe', 'Movie Theater', 'Lounge', 'Boutique',
       'Coffee Shop', 'Salon / Barbershop', 'Other Nightlife',
       'Residential Building (Apartment / Condo)', 'Playground',
       'Pizza Place', 'Theme Park Ride / Attraction',
       'General Entertainment', 'Office', 'Electronics Store', 'Church',
       'Planetarium', 'Jewelry Store', 'Factory',
       'Financial or Legal Service', 'Video Store', 'Laundromat',
       'Home Service', 'Gift Shop', 'Distribution Center',
       'Thrift / Vintage Store', 'Event Service', 'Video Game Store'],
      dtype=object)

In [28]:
ny_sorted = ny_sorted[ny_sorted['Category_Name'] != 'Clothing Store']
ny_sorted = ny_sorted[ny_sorted['Category_Name'] != 'Road']
ny_sorted = ny_sorted[ny_sorted['Category_Name'] != 'Boutique']
ny_sorted = ny_sorted[ny_sorted['Category_Name'] != 'Coffee Shop']
ny_sorted = ny_sorted[ny_sorted['Category_Name'] != 'Salon / Barbershop']
ny_sorted = ny_sorted[ny_sorted['Category_Name'] != 'Church']
ny_sorted = ny_sorted[ny_sorted['Category_Name'] != 'Planetarium']
ny_sorted = ny_sorted[ny_sorted['Category_Name'] != 'Jewelry Store']
ny_sorted = ny_sorted[ny_sorted['Category_Name'] != 'Factory']
ny_sorted = ny_sorted[ny_sorted['Category_Name'] != 'Financial or Legal Service']
ny_sorted = ny_sorted[ny_sorted['Category_Name'] != 'Video Store']
ny_sorted = ny_sorted[ny_sorted['Category_Name'] != 'Laundromat']
ny_sorted = ny_sorted[ny_sorted['Category_Name'] != 'Home Service']
ny_sorted = ny_sorted[ny_sorted['Category_Name'] != 'Gift Shop']
ny_sorted = ny_sorted[ny_sorted['Category_Name'] != 'Distribution Center']
ny_sorted = ny_sorted[ny_sorted['Category_Name'] != 'Thrift / Vintage Store']
#it isn't clear if the other categories aren't arcades
ny_sorted['Category_Name'].unique()

array(['Bowling Alley', 'Arcade', 'Building', 'Plaza', 'Gaming Cafe',
       'Movie Theater', 'Lounge', 'Other Nightlife',
       'Residential Building (Apartment / Condo)', 'Playground',
       'Pizza Place', 'Theme Park Ride / Attraction',
       'General Entertainment', 'Office', 'Electronics Store',
       'Event Service', 'Video Game Store'], dtype=object)

In [29]:
ny_sorted.to_csv('{}_sorted.csv'.format(search_query))

### Obtaining Neighborhood for Each Arcade

In [30]:
ny_sorted = pd.read_csv('{}_sorted.csv'.format(search_query))

In [31]:
ny_sorted = ny_sorted.drop('Unnamed: 0',axis =1 )

In [33]:
print(len(ny_sorted))
ny_sorted.head(30)

326


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Clinton,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,340,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley
1,Hudson Yards,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,605,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley
2,Midtown,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,992,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley
3,Midtown South,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,1030,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley
4,Murray Hill,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,1581,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley
5,Chelsea,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,1696,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley
6,Lincoln Square,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,1929,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley
7,Flatiron,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,1957,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley
8,Tudor City,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,2160,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley
9,Turtle Bay,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,2205,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley


In [34]:
count = -1
list_to_drop = list()
for ID in ny_sorted['Venue_ID']:
    count = count +1
    try:
        if ny_sorted['Venue_ID'][count] == ny_sorted['Venue_ID'][count-1] and ny_sorted['Venue_Distance'][count]>=ny_sorted['Venue_Distance'][count-1]:
            list_to_drop.append(count)
    except:
        continue
list_to_drop

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 25,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 47,
 48,
 49,
 50,
 51,
 53,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 64,
 65,
 66,
 67,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 101,
 102,
 103,
 105,
 106,
 107,
 108,
 109,
 110,
 112,
 113,
 114,
 115,
 116,
 117,
 119,
 121,
 122,
 123,
 124,
 125,
 126,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 192,
 193,
 194,
 195,
 196,
 198,
 199,
 200,
 201,
 203,
 204,
 205,
 206,


In [35]:
ny_final = ny_sorted.drop(index = list_to_drop)

In [36]:
print(len(ny_final))
ny_final

39


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Clinton,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,340,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley
10,Murray Hill,42869100f964a52018231fe3,5th ave. arcade inc.,348,260 5th Ave,40.746602,-73.981801,4bf58dd8d48988d1e1931735,Arcade
24,Coney Island,49ebfb9df964a5204d671fe3,Eldorado Bumper Cars & Arcade,725,1216 Surf Ave,40.575221,-73.980188,4bf58dd8d48988d1e1931735,Arcade
26,Midtown South,4a326ac3f964a520499a1fe3,Kaufman Arcade,300,139 W 35th Street,40.751193,-73.989083,4bf58dd8d48988d130941735,Building
37,Upper East Side,4b996827f964a520377935e3,Central Park - The Arcade,902,Central Park,40.773860,-73.970952,4bf58dd8d48988d164941735,Plaza
46,Ocean Hill,4bf9ff045ec320a12d298bd3,Arcade Restaurant,362,No Address,40.681484,-73.911668,4bf58dd8d48988d130941735,Building
52,Coney Island,4c2e9b5b09a99c746c9c0a2a,Deno's Arcade,818,1025 Boardwalk W,40.573882,-73.979015,4bf58dd8d48988d1e1931735,Arcade
54,Prospect Heights,4c4a1cb69c8d2d7fb1be1069,At the arcade,369,No Address,40.678681,-73.968486,4bf58dd8d48988d1e1931735,Arcade
63,Central Harlem,4c649a46e0c4be9aca168658,Harlem Arcade,160,No Address,40.816290,-73.941351,4bf58dd8d48988d1e1931735,Arcade
68,Little Italy,4cbf45eb28d176b0b0ef226e,Moto Arcade,262,201 Mulberry St,40.721616,-73.996588,4bf58dd8d48988d11a941735,Other Nightlife


In [37]:
ny_final.to_csv('{}_unique.csv'.format(search_query))

### Pulling the Ratings

In [38]:
search_query

'Arcade'

In [39]:
ny_final = pd.read_csv('{}_unique.csv'.format(search_query))
ny_final.head()

,Unnamed: 0,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,0,Clinton,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,340,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley
1,10,Murray Hill,42869100f964a52018231fe3,5th ave. arcade inc.,348,260 5th Ave,40.746602,-73.981801,4bf58dd8d48988d1e1931735,Arcade
2,24,Coney Island,49ebfb9df964a5204d671fe3,Eldorado Bumper Cars & Arcade,725,1216 Surf Ave,40.575221,-73.980188,4bf58dd8d48988d1e1931735,Arcade
3,26,Midtown South,4a326ac3f964a520499a1fe3,Kaufman Arcade,300,139 W 35th Street,40.751193,-73.989083,4bf58dd8d48988d130941735,Building
4,37,Upper East Side,4b996827f964a520377935e3,Central Park - The Arcade,902,Central Park,40.773860,-73.970952,4bf58dd8d48988d164941735,Plaza


In [40]:
ny_final = ny_final.drop('Unnamed: 0', axis = 1)
ny_final.head()

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Clinton,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,340,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley
1,Murray Hill,42869100f964a52018231fe3,5th ave. arcade inc.,348,260 5th Ave,40.746602,-73.981801,4bf58dd8d48988d1e1931735,Arcade
2,Coney Island,49ebfb9df964a5204d671fe3,Eldorado Bumper Cars & Arcade,725,1216 Surf Ave,40.575221,-73.980188,4bf58dd8d48988d1e1931735,Arcade
3,Midtown South,4a326ac3f964a520499a1fe3,Kaufman Arcade,300,139 W 35th Street,40.751193,-73.989083,4bf58dd8d48988d130941735,Building
4,Upper East Side,4b996827f964a520377935e3,Central Park - The Arcade,902,Central Park,40.773860,-73.970952,4bf58dd8d48988d164941735,Plaza


In [41]:
count = -1
rating_list = list()
for ident in ny_final['Venue_ID']:
    count = count +1
    venue_id = ny_final['Venue_ID'][count]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
    result = requests.get(url).json()
    try:
        rating = result['response']['venue']['rating']
    except:
        rating = 'Not Rated'
    rating_list.append(rating)
rating_list

[6.8,
 'Not Rated',
 8.3,
 'Not Rated',
 8.8,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 6.7,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 7.5,
 7.5,
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated']

In [43]:
ny_final = ny_final.assign(Venue_Rating = rating_list)

In [44]:
ny_final.head()

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name,Venue_Rating
0,Clinton,3fd66200f964a5200ae91ee3,Frames Bowling Lounge,340,550 9th Ave,40.757187,-73.992965,4bf58dd8d48988d1e4931735,Bowling Alley,6.8
1,Murray Hill,42869100f964a52018231fe3,5th ave. arcade inc.,348,260 5th Ave,40.746602,-73.981801,4bf58dd8d48988d1e1931735,Arcade,Not Rated
2,Coney Island,49ebfb9df964a5204d671fe3,Eldorado Bumper Cars & Arcade,725,1216 Surf Ave,40.575221,-73.980188,4bf58dd8d48988d1e1931735,Arcade,8.3
3,Midtown South,4a326ac3f964a520499a1fe3,Kaufman Arcade,300,139 W 35th Street,40.751193,-73.989083,4bf58dd8d48988d130941735,Building,Not Rated
4,Upper East Side,4b996827f964a520377935e3,Central Park - The Arcade,902,Central Park,40.773860,-73.970952,4bf58dd8d48988d164941735,Plaza,8.8


In [46]:
ny_final.to_csv('{}_final_clean.csv'.format(search_query))